<a href="https://colab.research.google.com/github/AryanSai/ASR/blob/main/Indic_Conformer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

### Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-kdx0tp5j/nemo-toolkit_e8dcb2d25f834b1b9ab7428e1067b06d
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-kdx0tp5j/nemo-toolkit_e8dcb2d25f834b1b9ab7428e1067b06d
  Resolved https://github.com/NVIDIA/NeMo.git to commit 95ca2f45034447ecd11bf29a0ab55d9079133db1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 8

In [3]:
import nemo.collections.asr as nemo_asr

In [4]:
hf_model_name = 'parthiv11/stt_hi_conformer_ctc_large_v2'
hf_model = nemo_asr.models.ASRModel.from_pretrained(hf_model_name)

[NeMo W 2024-06-06 10:41:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
    The secret `HF_TOKEN` does not exist in your Colab secrets.
    To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
    You will be able to reuse this secret in all of your notebooks.
    Please note that authentication is recommended but still optional to access public models or datasets.
      warnings.warn(
    


stt_hi_conformer_ctc_large_v2.nemo:   0%|          | 0.00/461M [00:00<?, ?B/s]

[NeMo I 2024-06-06 10:41:43 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2024-06-06 10:41:44 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /mnt/megh/hindi/manifest/asr_set_03/train/train.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30.0
    min_duration: 0.5
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-06-06 10:41:44 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /mnt/megh/hindi/manifest/asr_set_03/

[NeMo I 2024-06-06 10:41:44 features:305] PADDING: 0
[NeMo I 2024-06-06 10:41:46 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--parthiv11--stt_hi_conformer_ctc_large_v2/snapshots/fe8f550d16ea4f29cdbfc12924e4a3d8adbb36af/stt_hi_conformer_ctc_large_v2.nemo.


In [5]:
import torch
import torchaudio
import nemo.collections.asr as nemo_asr
from concurrent.futures import ThreadPoolExecutor



# Function for transcribing audio
def transcribe_single_audio(audio_file, model=hf_model, device='cuda'):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(audio_file)

    # Resample the audio if needed
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Convert stereo to mono if needed
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Perform inference
    with torch.no_grad():
        # Ensure the model and input are on the same device
        waveform = waveform.to(device)
        model = model.to(device)
        logits, logits_len, _ = model.forward(input_signal=waveform, input_signal_length=torch.tensor([waveform.size(1)]).to(device))
        current_hypotheses, _ = model.decoding.ctc_decoder_predictions_tensor(
            logits, decoder_lengths=logits_len, return_hypotheses=True,
        )
        text = current_hypotheses[0].text
    return text

def transcribe_audio(audio_files, model=hf_model, device='cuda', max_workers=None):

    # Transcribe each audio file in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit transcription tasks
        futures = [executor.submit(transcribe_single_audio, audio_file, model, device) for audio_file in audio_files]

        # Collect results
        results = [future.result() for future in futures]

    return results

In [ ]:
# download your audio files

In [6]:
transcription=transcribe_audio(["/content/try2.wav"])

[NeMo W 2024-06-06 10:42:08 ctc_greedy_decoding:168] CTC decoding strategy 'greedy' is slower than 'greedy_batch', which implements the same exact interface. Consider changing your strategy to 'greedy_batch' for a free performance improvement.


In [7]:
transcription

['अओ श्री महा गणाधिन वर बीर ही अधर तो सान कसम मेत यम चदिथकाम द प्रारंभाो यहज एक नानंरा धमम बोरस्ता दसी मतलब सुरू जेनह सब दी उपमा अस्य विषठा सातस्तिवगा']